In [2]:
import sqlite3, pandas as pd
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# Question 1 : Explain the fundamental differences between DDL, DML, and DQL
# commands in SQL. Provide one example for each type of command.

Answer :
# DDL defines and manages the structure of database objects.
# Example DDL:
"""
CREATE TABLE Employees (
  EmployeeID INT PRIMARY KEY,
  Name VARCHAR(100),
  HireDate DATE
);
"""
# DML manipulates data within those structures.
# Example DML:
"""
INSERT INTO Employees (EmployeeID, Name, HireDate)
VALUES (1,'Akash Kumar','2018-06-07');
UPDATE Employees SET Name='Akash K.' WHERE EmployeeID=1;
DELETE FROM Employees WHERE EmployeeID=1;
"""
# DQL retrieves data for analysis.
# Example DQL:
"""
SELECT EmployeeID, Name, HireDate
FROM Employees
WHERE HireDate >= '2018-01-01';
"""

# Question 2 : What is the purpose of SQL constraints? Name and describe
# three common types of constraints, providing a simple scenario
# where each would be useful.
Answer:
# Constraints maintain data accuracy and consistency.
# Common constraints: PRIMARY KEY, UNIQUE, FOREIGN KEY.

# Example PRIMARY KEY:
"""
CREATE TABLE Customers (
  CustomerID INT PRIMARY KEY,
  CustomerName VARCHAR(100)
);
"""

# Example UNIQUE:
"""
CREATE TABLE Users (
  UserID INT PRIMARY KEY,
  Email VARCHAR(255) UNIQUE
);
"""

# Example FOREIGN KEY:
"""
CREATE TABLE Categories (
  CategoryID INT PRIMARY KEY,
  CategoryName VARCHAR(50)
);
CREATE TABLE Products (
  ProductID INT PRIMARY KEY,
  ProductName VARCHAR(100),
  CategoryID INT,
  FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)
);
"""

In [11]:
# Question 3 :  Explain the difference between LIMIT and OFFSET clauses in SQL.
# How would you use them together to retrieve the third page of results,
# assuming each page has 10 records?

# LIMIT restricts how many rows are returned.
# OFFSET skips a specific number of rows before returning results.
# For the third page (10 records per page): OFFSET = (3-1)*10 = 20.

cur.execute("CREATE TABLE SampleData1 (ID INTEGER PRIMARY KEY, Name TEXT)")
cur.executemany("INSERT INTO SampleData (Name) VALUES (?)", [(f'Record_{i}',) for i in range(1,31)])
page_number, page_size = 3, 10
offset_value = (page_number-1)*page_size
q3 = f"SELECT * FROM SampleData ORDER BY ID LIMIT {page_size} OFFSET {offset_value};"
print("\nQ3 Output - Third page of results (records 21-30):")
print(pd.read_sql_query(q3, conn))


Q3 Output - Third page of results (records 21-30):
   ID       Name
0  21  Record_21
1  22  Record_22
2  23  Record_23
3  24  Record_24
4  25  Record_25
5  26  Record_26
6  27  Record_27
7  28  Record_28
8  29  Record_29
9  30  Record_30


# Question 4 : What is a Common Table Expression (CTE) in SQL, and what are
# its main benefits? Provide a simple SQL example demonstrating its usage.
Answer:
# A CTE is a temporary named result set defined with WITH.
# It makes complex queries easier to read and supports recursion.

"""
WITH RecentOrders AS (
  SELECT OrderID, CustomerID, OrderDate
  FROM Orders
  WHERE OrderDate >= '2023-01-01'
)
SELECT CustomerID, COUNT(*) AS NumOrders
FROM RecentOrders
GROUP BY CustomerID;
"""

# Question 5 : Describe the concept of SQL Normalization and its primary goals.
# Briefly explain the first three normal forms (1NF, 2NF, 3NF).
Answer:
# Normalization organizes data to reduce redundancy and improve integrity.
# 1NF: Each column holds atomic values only.
# 2NF: Table is in 1NF and all non-key columns depend on the full primary key.
# 3NF: Table is in 2NF and non-key columns do not depend on other non-key columns.

# Example:
# 1NF - separate multiple phone numbers into new rows.
# 2NF - move ProductName to Products table if key=(OrderID,ProductID).
# 3NF - move CityName to Cities table if CityID → CityName.

In [3]:
# Question 6 :  Create a database named ECommerceDB and perform the following tasks:
# (tables, datatypes, constraints, and inserts as given)

cur.executescript("""
CREATE TABLE Categories (
  CategoryID INTEGER PRIMARY KEY,
  CategoryName VARCHAR(50) NOT NULL UNIQUE
);
CREATE TABLE Products (
  ProductID INTEGER PRIMARY KEY,
  ProductName VARCHAR(100) NOT NULL UNIQUE,
  CategoryID INTEGER,
  Price NUMERIC NOT NULL,
  StockQuantity INTEGER,
  FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)
);
CREATE TABLE Customers (
  CustomerID INTEGER PRIMARY KEY,
  CustomerName VARCHAR(100) NOT NULL,
  Email VARCHAR(100) UNIQUE,
  JoinDate DATE
);
CREATE TABLE Orders (
  OrderID INTEGER PRIMARY KEY,
  CustomerID INTEGER,
  OrderDate DATE NOT NULL,
  TotalAmount NUMERIC,
  FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

INSERT INTO Categories VALUES
(1,'Electronics'),(2,'Books'),(3,'Home Goods'),(4,'Apparel');

INSERT INTO Products VALUES
(101,'Laptop Pro',1,1200.00,50),
(102,'SQL Handbook',2,45.50,200),
(103,'Smart Speaker',1,99.99,150),
(104,'Coffee Maker',3,75.00,80),
(105,'Novel : The Great SQL',2,25.00,120),
(106,'Wireless Earbuds',1,150.00,100),
(107,'Blender X',3,120.00,60),
(108,'T-Shirt Casual',4,20.00,300);

INSERT INTO Customers VALUES
(1,'Alice Wonderland','alice@example.com','2023-01-10'),
(2,'Bob the Builder','bob@example.com','2022-11-25'),
(3,'Charlie Chaplin','charlie@example.com','2023-03-01'),
(4,'Diana Prince','diana@example.com','2021-04-26');

INSERT INTO Orders VALUES
(1001,1,'2023-04-26',1245.50),
(1002,2,'2023-10-12',99.99),
(1003,1,'2023-07-01',145.00),
(1004,3,'2023-01-14',150.00),
(1005,2,'2023-09-24',120.00),
(1006,1,'2023-06-19',20.00);
""")
print("\nQ6: Tables created and data inserted successfully.")


Q6: Tables created and data inserted successfully.


In [6]:
# Question 7 : Generate a report showing CustomerName, Email, and the
# TotalNumberofOrders for each customer. Include customers who have not placed
# any orders, in which case their TotalNumberofOrders should be 0.
# Order the results by CustomerName.

q7 = """
SELECT c.CustomerName, c.Email, COALESCE(COUNT(o.OrderID),0) AS TotalOrders
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID=o.CustomerID
GROUP BY c.CustomerID
ORDER BY c.CustomerName;
"""
print("\nQ7 Output - Total Orders per Customer:")
print(pd.read_sql_query(q7, conn))


Q7 Output - Total Orders per Customer:
       CustomerName                Email  TotalOrders
0  Alice Wonderland    alice@example.com            3
1   Bob the Builder      bob@example.com            2
2   Charlie Chaplin  charlie@example.com            1
3      Diana Prince    diana@example.com            0


In [12]:
# Question 8 :  Retrieve Product Information with Category: Write a SQL query to
# display the ProductName, Price, StockQuantity, and CategoryName for all products.
# Order the results by CategoryName and then ProductName alphabetically.

q8 = """
SELECT p.ProductName, p.Price, p.StockQuantity, c.CategoryName
FROM Products p
JOIN Categories c ON p.CategoryID=c.CategoryID
ORDER BY c.CategoryName, p.ProductName;
"""
print("\nQ8 Output - Product Information with Category:")
print(pd.read_sql_query(q8, conn))


Q8 Output - Product Information with Category:
             ProductName    Price  StockQuantity CategoryName
0         T-Shirt Casual    20.00            300      Apparel
1  Novel : The Great SQL    25.00            120        Books
2           SQL Handbook    45.50            200        Books
3             Laptop Pro  1200.00             50  Electronics
4          Smart Speaker    99.99            150  Electronics
5       Wireless Earbuds   150.00            100  Electronics
6              Blender X   120.00             60   Home Goods
7           Coffee Maker    75.00             80   Home Goods


In [13]:
# Question 9 : Write a SQL query that uses a Common Table Expression (CTE) and a
# Window Function (specifically ROW_NUMBER() or RANK()) to display the
# CategoryName, ProductName, and Price for the top 2 most expensive products
# in each CategoryName.

q9 = """
WITH Ranked AS (
  SELECT c.CategoryName, p.ProductName, p.Price,
         ROW_NUMBER() OVER(PARTITION BY c.CategoryName ORDER BY p.Price DESC) AS rn
  FROM Products p
  JOIN Categories c ON p.CategoryID=c.CategoryID
)
SELECT CategoryName, ProductName, Price
FROM Ranked
WHERE rn <= 2
ORDER BY CategoryName, Price DESC;
"""
print("\nQ9 Output - Top 2 Most Expensive Products per Category:")
print(pd.read_sql_query(q9, conn))


Q9 Output - Top 2 Most Expensive Products per Category:
  CategoryName            ProductName   Price
0      Apparel         T-Shirt Casual    20.0
1        Books           SQL Handbook    45.5
2        Books  Novel : The Great SQL    25.0
3  Electronics             Laptop Pro  1200.0
4  Electronics       Wireless Earbuds   150.0
5   Home Goods              Blender X   120.0
6   Home Goods           Coffee Maker    75.0


In [14]:
# ================================================================
# Question 10 : You are hired as a data analyst by Sakila Video Rentals, a global movie
# rental company. The management team is looking to improve decision-making by
# analyzing existing customer, rental, and inventory data.
#
# Using the Sakila database, answer the following business questions:
# 1. Identify the top 5 customers based on the total amount they’ve spent.
#    Include customer name, email, and total amount spent.
# 2. Which 3 movie categories have the highest rental counts?
#    Display the category name and number of times movies from that category were rented.
# 3. Calculate how many films are available at each store and how many of those
#    have never been rented.
# 4. Show the total revenue per month for the year 2023 to analyze business seasonality.
# 5. Identify customers who have rented more than 10 times in the last 6 months.
# ================================================================

# The following queries are written for the Sakila database (MySQL or SQLite equivalent).

q10_1 = """
SELECT c.customer_id,
       c.first_name||' '||c.last_name AS customer_name,
       c.email,
       SUM(p.amount) AS total_spent
FROM customer c
JOIN payment p ON c.customer_id=p.customer_id
GROUP BY c.customer_id
ORDER BY total_spent DESC
LIMIT 5;
"""

q10_2 = """
SELECT cat.name AS category_name, COUNT(r.rental_id) AS rental_count
FROM category cat
JOIN film_category fc ON cat.category_id=fc.category_id
JOIN inventory i ON fc.film_id=i.film_id
JOIN rental r ON i.inventory_id=r.inventory_id
GROUP BY cat.category_id
ORDER BY rental_count DESC
LIMIT 3;
"""

q10_3 = """
SELECT s.store_id, COUNT(i.inventory_id) AS total_films,
       SUM(CASE WHEN r.rental_id IS NULL THEN 1 ELSE 0 END) AS never_rented
FROM store s
JOIN inventory i ON s.store_id=i.store_id
LEFT JOIN rental r ON i.inventory_id=r.inventory_id
GROUP BY s.store_id;
"""

q10_4 = """
SELECT STRFTIME('%Y-%m', p.payment_date) AS year_month,
       SUM(p.amount) AS total_revenue
FROM payment p
WHERE STRFTIME('%Y', p.payment_date)='2023'
GROUP BY year_month
ORDER BY year_month;
"""

q10_5 = """
SELECT c.customer_id,
       c.first_name||' '||c.last_name AS customer_name,
       c.email,
       COUNT(r.rental_id) AS rental_count
FROM customer c
JOIN rental r ON c.customer_id=r.customer_id
WHERE date(r.rental_date)>=date('now','-6 months')
GROUP BY c.customer_id
HAVING COUNT(r.rental_id)>10
ORDER BY rental_count DESC;
"""

print("\nQ10 - Sakila Analytical SQL Queries (for execution on Sakila database):")
print(q10_1)
print(q10_2)
print(q10_3)
print(q10_4)
print(q10_5)


Q10 - Sakila Analytical SQL Queries (for execution on Sakila database):

SELECT c.customer_id,
       c.first_name||' '||c.last_name AS customer_name,
       c.email,
       SUM(p.amount) AS total_spent
FROM customer c
JOIN payment p ON c.customer_id=p.customer_id
GROUP BY c.customer_id
ORDER BY total_spent DESC
LIMIT 5;


SELECT cat.name AS category_name, COUNT(r.rental_id) AS rental_count
FROM category cat
JOIN film_category fc ON cat.category_id=fc.category_id
JOIN inventory i ON fc.film_id=i.film_id
JOIN rental r ON i.inventory_id=r.inventory_id
GROUP BY cat.category_id
ORDER BY rental_count DESC
LIMIT 3;


SELECT s.store_id, COUNT(i.inventory_id) AS total_films,
       SUM(CASE WHEN r.rental_id IS NULL THEN 1 ELSE 0 END) AS never_rented
FROM store s
JOIN inventory i ON s.store_id=i.store_id
LEFT JOIN rental r ON i.inventory_id=r.inventory_id
GROUP BY s.store_id;


SELECT STRFTIME('%Y-%m', p.payment_date) AS year_month,
       SUM(p.amount) AS total_revenue
FROM payment p
WHERE ST